In [1]:
!nvcc --version
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118 --quiet

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 988.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.0+cu118 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.0+cu118 which is i

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# put path to audio.zip in google drive here:

Mounted at /content/drive


In [3]:
# make the appropriate folders to keep track of stuff

# adapted from DCASE 2023 Task 7 starter code, found here:
# https://colab.research.google.com/drive/1FzbBf_FqWKu59i97ITibJbdPAqSzeMD4?usp=sharing
%cd '/content/'
!mkdir lpcc
%cd '/content/lpcc'
# clone edited fork of basecode
!git clone https://github.com/arililoia-cmu/lp-music-caps.git .



/content
/content/lpcc
Cloning into '.'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 219 (delta 94), reused 178 (delta 57), pack-reused 0
Receiving objects: 100% (219/219), 3.38 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [4]:
%pip install -e .

Obtaining file:///content/lpcc
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00

In [1]:
#replace with path to audio zip in your own google drive
audio_zip_path = '/content/drive/MyDrive/wavcaps stuff/audiocopy.zip'
%cd '/content/lpcc'

/content/lpcc


In [2]:
import os

final_dir = '/content/lpcc'
!mkdir test_set

get_file_id_from_google_drive_link = lambda file_link:file_link.split('/d/')[-1].split('/view')[0]
get_wget_command = lambda file_id,save_name: f'wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id={file_id}" -O- | sed -rn "s/.*confirm=([0-9A-Za-z_]+).*/\1\n/p")&id={file_id}" -O {save_name} && rm -rf ~/cookies.txt'

download_files_google_drive_link_list = [
    ('song_tags.csv','https://drive.google.com/file/d/1YYBxMD45hGGEo92m1iEo1KsbGU2CXcKp/view?usp=sharing'),
    ('split.json', 'https://drive.google.com/file/d/1eO2mmUz3WAlr1DEwcswY8mlB4DtBSqJc/view?usp=sharing')
]
for google_drive_link in download_files_google_drive_link_list:
  print(f'downloading: {google_drive_link[0]}')
  os.system(get_wget_command(get_file_id_from_google_drive_link(google_drive_link[1]),f'{final_dir}/{google_drive_link[0]}'))



mkdir: cannot create directory ‘test_set’: File exists
downloading: song_tags.csv
downloading: split.json


In [3]:
#running app.py to download the model from huggingface

%cd '/content/lpcc/demo'
!python app.py

/content/lpcc/demo
2023-10-30 19:27:46.067696: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 19:27:46.067758: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 19:27:46.067789: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 19:27:46.076692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 19:27:47.358

In [4]:
#app/demo.py

import os
import argparse
import gradio as gr
from timeit import default_timer as timer
import torch
import numpy as np
import pandas as pd
from huggingface_hub import hf_hub_download
from model.bart import BartCaptionModel
from utils.audio_utils import load_audio, STR_CH_FIRST
import string

if os.path.isfile("transfer.pth") == False:
    torch.hub.download_url_to_file('https://huggingface.co/seungheondoh/lp-music-caps/resolve/main/transfer.pth', 'transfer.pth')

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = BartCaptionModel(max_length = 128)
pretrained_object = torch.load('./transfer.pth', map_location='cpu')
state_dict = pretrained_object['state_dict']
model.load_state_dict(state_dict)
if torch.cuda.is_available():
    torch.cuda.set_device(device)
model = model.cuda(device)
model.eval()


BartCaptionModel(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, b

In [5]:

def get_audio(audio_path, duration=10, target_sr=16000):
    n_samples = int(duration * target_sr)
    audio, sr = load_audio(
        path= audio_path,
        ch_format= STR_CH_FIRST,
        sample_rate= target_sr,
        downmix_to_mono= True,
    )
    if len(audio.shape) == 2:
        audio = audio.mean(0, False)  # to mono
    input_size = int(n_samples)
    if audio.shape[-1] < input_size:  # pad sequence
        pad = np.zeros(input_size)
        pad[: audio.shape[-1]] = audio
        audio = pad
    ceil = int(audio.shape[-1] // n_samples)
    audio = torch.from_numpy(np.stack(np.split(audio[:ceil * n_samples], ceil)).astype('float32'))
    return audio

def captioning(audio_path):
    audio_tensor = get_audio(audio_path = audio_path)
    if device is not None:
        audio_tensor = audio_tensor.to(device)
    with torch.no_grad():
        output = model.generate(
            samples=audio_tensor,
            num_beams=5,
        )

    inference = []
    number_of_chunks = range(audio_tensor.shape[0])
    for chunk, text in zip(number_of_chunks, output):
        # time = f"[{chunk * 10}:00-{(chunk + 1) * 10}:00]"

        # text_no_punctuation = text.translate(str.maketrans('', '', string.punctuation))
        inference.append(text)
    return inference



In [6]:
from bert_score import score

In [7]:
%cd '/content/lpcc/demo'
!mkdir wavfiles
destination_directory = '/content/lpcc/demo/wavfiles'

/content/lpcc/demo


In [9]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
gc.collect()
gc.collect()
gc.collect()

0

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
from datasets import load_dataset
pseudo_captions_dataset = load_dataset("seungheondoh/LP-MusicCaps-MSD")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [12]:
pcd_test = pseudo_captions_dataset['test']
print(len(pcd_test))
print(pcd_test[0])

# get (in order) concatenated tags, caption_writing, caption_summary,
# caption_paraphrase, caption_attribute_prediction
def get_caption_from_track_id(search_value_track_ID):
  # search_value_track_ID = "TROLQDY128F429F80F"
  matching_entry = next((entry for entry in pcd_test if entry["track_id"] == search_value_track_ID), None)
  if matching_entry:
      # concatenated tags:
      to_return = []
      value_taglist = matching_entry["tag"]
      value_taglist_concat = ", ".join(value_taglist)
      to_return.append(value_taglist_concat)
      # caption_writing
      to_return.append(matching_entry["caption_writing"])
      to_return.append(matching_entry["caption_summary"])
      to_return.append(matching_entry["caption_paraphrase"])
      to_return.append(matching_entry["caption_attribute_prediction"])
      return to_return
  else:
      return ["","","","",""]




34631
{'track_id': 'TREADPD128F933E680', 'title': 'Written in the stars', 'artist_name': "Blackmore's Night", 'release': 'Fires at midnight', 'year': 2001, 'tag': ['dramatic', 'guitar virtuoso', 'heavy metal', 'indulgent', 'pop rock', 'atmospheric', 'hard rock', 'refined', 'folk'], 'caption_writing': 'This song showcases the undeniable talent of a guitar virtuoso, seamlessly blending the refined elements of pop rock and folk with the atmospheric and dramatic sounds of heavy metal and hard rock, resulting in an indulgent and unforgettable musical experience.', 'caption_summary': "This song showcases a guitar virtuoso's refined and atmospheric pop rock sound, with elements of dramatic heavy metal, folk, and indulgent hard rock.", 'caption_paraphrase': 'This song showcases the refined playing of a guitar virtuoso at the forefront of intricate pop rock arrangements, with atmospheric and dramatic elements that draw from heavy metal, folk, and indulgent hard rock influences.', 'caption_attri

In [14]:
import json
import zipfile
import gc
import csv

split_path = '/content/lpcc/split.json'

with open(split_path, 'r') as f:
  split = json.load(f)
  test_track = split['test_track'] + split['extra_track']
  print(test_track)

len_test_track = len(test_track)


file_path = "test_set_captioning.csv"
with zipfile.ZipFile(audio_zip_path, 'r') as zip_ref:
  with open(file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["track_id", "generated_caption", "tag", "caption_writing", "caption_summary", "caption_paraphrase", "caption_attribute_prediction"])
    len_test_track = len(test_track)
    for i in range(len_test_track):
      print(i)
      gc.collect()
      torch.cuda.empty_cache()
      file_to_extract = 'audio/' + test_track[i] + '.m4a'
      if file_to_extract in zip_ref.namelist():
        zip_ref.extract(file_to_extract, path=destination_directory)
        # print(f"{file_to_extract} has been extracted.")
        # one caption is generated per 10 seconds of audio, so multiple
        # captions may be generated for the same file
        generated_captions = captioning(destination_directory + '/' + file_to_extract)
        # there are four gpt-generated captions per sample in the dataset
        # and the tag list separated by commas is treated as a fifth
        five_groundtruth_captions = get_caption_from_track_id(test_track[i])

        # we calculate the bert score between the caption generated by the model
        # and all four captions.
        for j in range(len(generated_captions)):
            csv_line = [test_track[i], generated_captions[j], five_groundtruth_captions[0], five_groundtruth_captions[1], five_groundtruth_captions[2], five_groundtruth_captions[3], five_groundtruth_captions[4]]
            csv_writer.writerow(csv_line)
        if os.path.exists(destination_directory + '/' + file_to_extract):
            os.remove(destination_directory + '/' + file_to_extract)
            # print(f"File '{destination_directory + '/' + file_to_extract}' has been deleted.")
        else:
            print(f"File '{destination_directory + '/' + file_to_extract}' does not exist.")

      else:
        print(f"{file_to_extract} does not exist in the archive.")

zip_ref.close()

['TROLQDY128F429F80F', 'TRKYOEP128F92CDF08', 'TRPNJBY128F429F812', 'TRHQCNU128F9309166', 'TRPVIDE12903C9A7D8', 'TRNSGJG12903CAD843', 'TRYMDHX128F147EDFB', 'TRATNKT128F428A7B4', 'TRVUEXJ128F930C329', 'TRNTONF128F933E7EE', 'TRSRMCQ128F92FAD76', 'TRRFZYA128F933B264', 'TRTJGBX128EF365330', 'TRUYHAK128F92D674F', 'TRXXAFE128F429600B', 'TRZCCLG128F930B5A5', 'TRORPHN128E0797FC4', 'TRAYQTF12903CA717A', 'TRPFERE128F147228A', 'TRZLYYQ12903CE6133', 'TRKGJYB128F425AD4F', 'TRREEST12903CA719F', 'TRGVCLE128F9325088', 'TRDFRJY128F9350030', 'TRGPSWR128F92CA25D', 'TRIPJID128F147DC8D', 'TREHVWP128F92ED749', 'TRRYGXC128F422F73A', 'TROTNSL128F426CD3A', 'TRBXUFK128F4259FCC', 'TRTRXLP12903CBBEAC', 'TRUKWPK128F4264431', 'TRPUEKA128F4213BCD', 'TRCRZAM128F4289C77', 'TRRYWHH128F92C66CD', 'TRPLXTH128EF352D9B', 'TRKANGS12903D10159', 'TRAKHDN128F933A5B3', 'TRRRAWA128F425B0D9', 'TRFKCEM128F42A4E1E', 'TRQKPYX128F42597C1', 'TREPTIM128F4243460', 'TRSERVZ128E0784700', 'TROWGLA128F14650DB', 'TRRBRUA128F92DE815', 'TRTJUHN1

Exception ignored in: <function _xla_gc_callback at 0x7e7b21e19cf0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


1264
1265
1266


KeyboardInterrupt: ignored

In [ ]:
# !pip install tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=f015cd6829c15021a4240c72c11f73eeb26a649a87430770356480e7659e7c01
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [15]:
# # !pip install fastapi==0.90.1
# !pip freeze

In [ ]:
# !python app.py

Traceback (most recent call last):
  File "/content/lpmusiccaps/demo/app.py", line 9, in <module>
    from model.bart import BartCaptionModel
  File "/content/lpmusiccaps/demo/model/bart.py", line 5, in <module>
    from .modules import AudioEncoder
  File "/content/lpmusiccaps/demo/model/modules.py", line 5, in <module>
    import torchaudio
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/__init__.py", line 1, in <module>
    from . import (  # noqa: F401
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/_extension/__init__.py", line 49, in <module>
    _check_cuda_version()
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/_extension/utils.py", line 192, in _check_cuda_version
    raise RuntimeError(
RuntimeError: Detected that PyTorch and TorchAudio were compiled with different CUDA versions. PyTorch has CUDA version 12.1 whereas TorchAudio has CUDA version 11.8. Please install the TorchAudio version that matches your PyTorch version.
